Read existing OSM Schutzgebiete from geojson export
Read (updated) DAV Shapefile

compare to find new or changed polygons and create an epsg:4326 shape file only with changed/new polygons for import in JOSM

Note: due to some epsg:4326 <-> epsg:31468 reprojection issue the geojson slightly (~ 2 meters) deviates from the DAV shapes
-> fuzzy matching with intersection over union approach is taken

Deleted polygons are not yet detected

In [84]:
import fiona
import pyproj
from shapely.geometry import shape, mapping
from shapely.ops import transform
from shapely.geometry import Polygon

import gpxpy
import gpxpy.gpx
from rdp import rdp

import copy

def dpr(poly,epsilon=0.000007):
    coords = []
    for x,y in poly.exterior.coords:
        coords.append([x,y])
    print(len(coords))
    coords = rdp(coords, epsilon=epsilon)
    print(len(coords))
    return Polygon(coords)

# for tests
origShapeFile = 'E:/OSM/Schutzgebiete/Test/Soinwand-XC.geojson' # epsg:4326
newShapeFile = 'E:/OSM/Schutzgebiete/Test/Soinwand-DAV.shp' # epsg:31468

shapesUpdateFile = 'E:/OSM/Schutzgebiete/New/new-shapes-test.shp'

# project = pyproj.Transformer.from_proj(
#     pyproj.Proj(init='epsg:4326'), # source coordinate system
#     pyproj.Proj(init='epsg:31468')) # destination coordinate system

oldFeatures = []
with fiona.open(origShapeFile) as input:
    oldCrs = input.crs
    schema = input.schema
    for feat in input:
        if feat['geometry'] != None and len(feat['geometry']['coordinates'][0]) > 2:
            poly = shape(feat['geometry'])
            poly = dpr(poly)
            feat['geometry'] = mapping(poly)
            oldFeatures.append(feat)
size = len(oldFeatures);          
print(f"OSM-Gebiete in {origShapeFile}: {size} / {oldCrs['init']}")

57
54
OSM-Gebiete in E:/OSM/Schutzgebiete/Test/Soinwand-XC.geojson: 1 / epsg:4326


In [85]:
def dump_gpx(filename, poly):
    gpx = gpxpy.gpx.GPX()
    gpx_track = gpxpy.gpx.GPXTrack()
    gpx.tracks.append(gpx_track)
    gpx_segment = gpxpy.gpx.GPXTrackSegment()
    gpx_track.segments.append(gpx_segment)
    for x,y in poly.exterior.coords:
        gpx_segment.points.append(gpxpy.gpx.GPXTrackPoint(y, x))
    with open(filename, "w") as out:    
        out.write(gpx.to_xml())

In [86]:
# poly = dpr(shape(oldFeatures[0]['geometry']), 0.000005)
dump_gpx(origShapeFile+".gpx", shape(oldFeatures[0]['geometry']))

In [87]:
# import matplotlib.pyplot as plt
# poly = shape(oldFeatures[0]['geometry'])
# x,y = poly.exterior.xy
# plt.plot(x,y)

In [88]:
# read and 3D to 2D convert abnd reproject DAV shapefile

project = pyproj.Transformer.from_proj(
    pyproj.Proj(init='epsg:31468'), # source coordinate system
    pyproj.Proj(init='epsg:4326')) # destination coordinate system

newFeatures = []
with fiona.open(newShapeFile) as input:
    schema = input.schema
    crs = input.crs
    if crs['init'] != "epsg:31468":
        print(f"Bad CRS {crs['init']} in {newShapeFile}")
        exit

    driver = input.driver
    for feat in input:
        if feat['geometry'] != None:
            if len(feat['geometry']['coordinates']) > 1:
                # multipolygons - code to be improved...
                for pfeat in feat['geometry']['coordinates']:
                    try: # some are len(1) lists, some aren't ?!
                        poly = Polygon(pfeat[0])
                    except:
                        poly = Polygon(pfeat)
                    poly = transform(lambda x, y, z=None: (x, y), poly)
                    poly = transform(project.transform, poly)
                    feat2 = copy.deepcopy(feat)
                    feat2['geometry'] = mapping(dpr(poly))
                    newFeatures.append(feat2)
                continue
            if len(feat['geometry']['coordinates'][0]) < 3:
                print("Skipping 2-point line")
                continue
            # transform 3D to 2D
            poly = shape(feat['geometry'])
            poly = transform(lambda x, y, z=None: (x, y), poly)
            poly = transform(project.transform, poly)
            feat['geometry'] = mapping(dpr(poly))
            newFeatures.append(feat)

size = len(newFeatures);          
print(f"Gebiete in {newShapeFile}: {size}")

58
54
Gebiete in E:/OSM/Schutzgebiete/Test/Soinwand-DAV.shp: 1


In [89]:
# poly = dpr(shape(newFeatures[0]['geometry']), 0.000005)
dump_gpx(newShapeFile+".gpx", shape(newFeatures[0]['geometry']))

In [90]:
# import matplotlib.pyplot as plt
# poly = shape(newFeatures[0]['geometry'])
# x,y = poly.exterior.xy
# plt.plot(x,y)

In [91]:
from shapely.geometry.polygon import orient

def normalize(polygon):
    
    def normalize_ring(ring):
        coords = ring.coords[:-1]
        start_index = min(range(len(coords)), key=coords.__getitem__)
        return coords[start_index:] + coords[:start_index]

    polygon = orient(polygon)
    normalized_exterior = normalize_ring(polygon.exterior)
    normalized_interiors = list(map(normalize_ring, polygon.interiors))
    return Polygon(normalized_exterior, normalized_interiors)


In [92]:
geod = pyproj.Geod(ellps='WGS84')
import statistics

# both polygons need to be in epsg:4326
def is_shifted(poly1, poly2):
    # poly1 = normalize(poly1)
    # poly2 = normalize(poly2)
    p1c = []
    p2c = []
    for x,y in poly1.exterior.coords:
        p1c.append([x,y])
    for x,y in poly2.exterior.coords:
        p2c.append([x,y])

    if len(p1c) != len(p2c):
        print(str(len(p1c))+" != "+str(len(p2c)))
        return False

    dists = []
    maxDist = 0
    for i in range(0,len(p2c)):
        azimuth1, azimuth2, distance = geod.inv(p1c[i][0], p1c[i][1], p2c[i][0], p2c[i][1])
        # print(i, p1c[i][0], p1c[i][1], p2c[i][0], p2c[i][1], distance)
        if distance > maxDist:
            maxDist = distance;
        dists.append(distance)

    print(f"maxDist={maxDist}")
    print("stdev="+str(statistics.stdev(dists)))
    print("var="+str(statistics.pvariance(dists)))

    if maxDist < 3.5 and statistics.stdev(dists) < 0.1 and statistics.pvariance(dists) < 0.005:
        print("shifted")
        return True
    else:
        return False

In [93]:
# https://www.reddit.com/r/gis/comments/mcw0y0/comparing_two_linestrings_with_shapely/

# from collections import OrderedDict
# dummyProps = OrderedDict([('Id', None),('Name', ''),('Regelung', '')])

newCount = 0 
foundCount = 0
shiftCount = 0
with fiona.open(shapesUpdateFile, 'w', crs={'init':'epsg:31468'}, driver='ESRI Shapefile', schema=schema) as out:
    for newFeature in newFeatures:
        # apply a buffer to avoid TopologyException: Input geom 1 is invalid: Self-intersection at or near point
        # https://www.programmersought.com/article/69515213493/
        newGeom = Polygon(shape(newFeature['geometry']).exterior)# .buffer(0.0001)
        found = False
        shifted = False
        for oldFeature in oldFeatures:
            try:
                oldGeom = Polygon(shape(oldFeature['geometry']))# .buffer(0.0001)
                shifted = is_shifted(newGeom, oldGeom)
                if shifted:
                    shiftCount += 1
            except Exception as ex:
                print(ex)
                print(oldFeature['geometry'])
                continue
            if not shifted:
                try:
                    # https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/
                    iou = newGeom.intersection(oldGeom).area / newGeom.union(oldGeom).area
                    print(iou)
                except Exception as ex:
                    print(ex)
                    print(newGeom)
                    print(oldGeom)
            if iou > 0.985 or shifted:
                found = True
                foundCount +=1
        if not found:
            newCount += 1
            out.write(newFeature)
print(f"Gefundene Gebiete = {foundCount}")
print(f"Shifted Gebiete = {shiftCount}")
print(f"Neue/Geänderte Gebiete = {newCount} -> in {shapesUpdateFile}")

maxDist=2.439906648536636
stdev=0.03267865758435603
var=0.0010481188344504864
shifted
Gefundene Gebiete = 1
Shifted Gebiete = 1
Neue/Geänderte Gebiete = 0 -> in E:/OSM/Schutzgebiete/New/new-shapes-test.shp
